In [ ]:
import geopandas as gpd

points = gpd.read_file("kappazunder-points.gpkg", layer="kappazunder_image_punkte")

In [ ]:
import numpy as np
from scipy.spatial import cKDTree
from tqdm import tqdm

# Load your points and ensure CRS is EPSG:31256 (meters)
gdf = points

# Extract coordinates as numpy array
coords = np.array(list(zip(gdf.geometry.x, gdf.geometry.y)))

# Shuffle indices for random selection
indices = np.arange(len(coords))
np.random.shuffle(indices)

# Build KDTree
tree = cKDTree(coords)

# Minimum distance threshold
min_dist = 20

# Boolean mask to track kept points
keep = np.ones(len(coords), dtype=bool)

for i in tqdm(indices):
    if keep[i]:
        # Find all points within min_dist of current point
        neighbors = tree.query_ball_point(coords[i], r=min_dist)
        # Mark all neighbors (except current point) as False
        neighbors.remove(i)
        keep[neighbors] = False

# Select sampled points
sampled_gdf = gdf[keep].copy()

# Save result
print(f"Original points: {len(gdf)}, Sampled points: {len(sampled_gdf)}")


100%|██████████| 1371422/1371422 [00:02<00:00, 470796.67it/s] 


Original points: 1371422, Sampled points: 111551


In [ ]:
gpd.to_file("sampled-points.gpkg", driver="GPKG", layer="kappazunder_image_punkte")